In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [3]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [4]:
val_df['Image_ID'].nunique()

56

In [5]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("zindi_challenge_cacao_stage2/train*/weights/best.pt"))[-1]
BEST_PATH = "zindi_challenge_cacao/train3/weights/best.pt"
BEST_PATH

'zindi_challenge_cacao/train3/weights/best.pt'

In [6]:
# Validate the model on the validation set
BEST_CFG = sorted(glob("zindi_challenge_cacao_stage2/train*/args.yaml"))[-1]
BEST_CFG = "zindi_challenge_cacao/train3/args.yaml"
BEST_CFG

'zindi_challenge_cacao/train3/args.yaml'

In [7]:
# Load the trained YOLO model
model = YOLO(BEST_PATH).eval()

In [8]:
for flag, v in ExifTags.TAGS.items():
	if v == "Orientation":
		break


def load_image_(filepath):
	image = Image.open(filepath)
	# return image

	exif = image._getexif()
	if exif is None:
		return image

	orientation_value = exif.get(flag, None)

	if orientation_value == 3:
		image = image.rotate(180, expand=True)
	elif orientation_value == 6:
		image = image.rotate(270, expand=True)
	elif orientation_value == 8:
		image = image.rotate(90, expand=True)
	return image

from ultralytics.utils.patches import imread
import cv2

def load_image(filepath):
	# return load_image_(filepath)
	return imread(filepath, cv2.IMREAD_COLOR)


flag

274

In [9]:
import yaml
with open(BEST_CFG, 'r') as f:
	cfg: dict = yaml.safe_load(f)
	print(cfg)

{'task': 'detect', 'mode': 'train', 'model': 'yolo11m.pt', 'data': 'data.yaml', 'epochs': 200, 'time': None, 'patience': 20, 'batch': 16, 'imgsz': 1024, 'save': True, 'save_period': -1, 'cache': False, 'device': 'cuda', 'workers': 8, 'project': 'zindi_challenge_cacao', 'name': 'train3', 'exist_ok': False, 'pretrained': True, 'optimizer': 'auto', 'verbose': True, 'seed': 0, 'deterministic': True, 'single_cls': False, 'rect': False, 'cos_lr': False, 'close_mosaic': 10, 'resume': False, 'amp': True, 'fraction': 1.0, 'profile': False, 'freeze': None, 'multi_scale': True, 'overlap_mask': True, 'mask_ratio': 4, 'dropout': 0.26323307173148563, 'val': True, 'split': 'val', 'save_json': False, 'conf': None, 'iou': 0.42361207118859606, 'max_det': 150, 'half': False, 'dnn': False, 'plots': True, 'source': None, 'vid_stride': 1, 'stream_buffer': False, 'visualize': False, 'augment': True, 'agnostic_nms': False, 'classes': None, 'retina_masks': False, 'embed': None, 'show': False, 'save_frames': Fa

In [10]:
# Batch size for predictions
batch_size = 16

cfg["device"] = "cuda:1"
cfg["batch"] = batch_size
cfg["conf"] = 0.0
cfg["verbose"] = False

cfg.pop("source", None)
# cfg.pop("batch_size")
cfg.pop("visualize", None)
cfg.pop("data", None)
cfg.pop("name", None)
cfg.pop("half", None)

cfg["model"] = "predict"

keys = list(cfg.keys())
for col in keys:
    if (
        "show" in col
        or "save" in col
        or "freeze" in col
        or "nms" in col
        or "multi_scale" in col
        or "plot" in col
        or "aug" in col
        or "drop" in col
    ):
        cfg.pop(col)

In [11]:
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = [i for i in os.listdir(test_dir_path) if not i.endswith(".npy")]

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []


with torch.no_grad():
	# Process images in batches
	for i in tqdm(range(0, len(image_files), batch_size)):
		batch_files = image_files[i:i + batch_size]
		batch_images = [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

		# Make predictions on the batch of images
		results = model.predict(
			batch_images,
			**cfg,
		)

		# Iterate through each result in the batch
		for img_file, result in zip(batch_files, results):
			boxes = result.boxes.xyxy.tolist() if result.boxes else []  # Bounding boxes in xyxy format
			classes = result.boxes.cls.tolist() if result.boxes else []  # Class indices
			confidences = result.boxes.conf.tolist() if result.boxes else []  # Confidence scores
			names = result.names  # Class names dictionary

			if boxes:  # If detections are found
				for box, cls, conf in zip(boxes, classes, confidences):
					x1, y1, x2, y2 = box
					detected_class = names[int(cls)]  # Get the class name from the names dictionary

					# Add the result to the all_data list
					all_data.append(
						{
							"Image_ID": str(img_file),
							"class": detected_class,
							"confidence": conf,
							"ymin": y1,
							"xmin": x1,
							"ymax": y2,
							"xmax": x2,
						}
					)
			else:  # If no objects are detected
				all_data.append(
					{
						"Image_ID": str(img_file),
						"class": "None",
						"confidence": None,
						"ymin": None,
						"xmin": None,
						"ymax": None,
						"xmax": None,
					}
				)


100%|██████████| 4/4 [00:06<00:00,  1.65s/it]


In [12]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [13]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_MYSxE2.jpg,healthy,0.739086,160.308838,387.705750,3210.924561,1925.129761
1,ID_MYSxE2.jpg,healthy,0.295568,0.353302,1809.238525,454.948883,2448.000000
2,ID_MYSxE2.jpg,healthy,0.051607,2788.655518,311.813141,3264.000000,1295.426392
3,ID_MYSxE2.jpg,healthy,0.023300,0.000000,1254.690918,529.274231,1809.436646
4,ID_MYSxE2.jpg,healthy,0.010546,0.000000,1204.025391,587.162170,2448.000000


In [14]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,8.400000e+03,8400.000000,8400.000000,8400.000000,8400.000000
mean,8.618134e-03,828.713862,680.097804,1205.334877,1031.435393
std,7.055673e-02,967.407787,783.112198,1082.173714,902.351843
min,7.105400e-07,0.000000,0.000000,0.000000,0.000000
25%,3.719463e-06,41.053033,26.458871,370.099510,332.250572
50%,1.230074e-05,499.103973,406.606552,939.622620,799.396210
75%,6.792816e-05,1198.054901,963.012268,1647.985992,1526.117615
max,8.520380e-01,4054.862305,3965.035400,4128.000000,4032.000000


In [15]:
sub['class'].value_counts()

class
healthy        3542
cssvd          2583
anthracnose    2275
Name: count, dtype: int64

In [16]:
def load_yolo_labels(label_folder):
	label_data = {}
	label_folder = Path(label_folder)
	paths = [i for i in label_folder.glob("*") if i.suffix != ".npy"]

	for label_file in paths:
		with open(label_file, "r") as file:
			annotations = []
			for line in file:
				parts = line.strip().split()
				if len(parts) == 5:
					class_id, x_center, y_center, width, height = map(float, parts)
					annotations.append({
						"class_id": int(class_id),
						"x_center": x_center,
						"y_center": y_center,
						"width": width,
						"height": height
					})
				else:
					print(f"Skipping line in {label_file}: {line.strip()}")
			label_data[label_file.stem] = annotations
	# Convert the label data to a pandas DataFrame
	label_df = []
	for image_id, annotations in label_data.items():
		for annotation in annotations:
			label_df.append({
				"Image_ID": image_id,
				"class_id": annotation["class_id"],
				"x_center": annotation["x_center"],
				"y_center": annotation["y_center"],
				"width": annotation["width"],
				"height": annotation["height"]
			})

	label_df = pd.DataFrame(label_df)
	return label_df

# Example usage
label_folder = VAL_LABELS_DIR
labels = load_yolo_labels(label_folder)
labels.sample(5)

def yolo_to_bbox(image_folder, labels_df: pd.DataFrame):
	image_folder = Path(image_folder)
	converted_bboxes = []

	paths = [i for i in image_folder.glob("*") if i.suffix != ".npy"]
	for image_file in paths:
		image_id = image_file.stem
		if image_id not in labels_df['Image_ID'].values:
			converted_bboxes.append({
				"Image_ID": image_id,
				"class_id": -1,  # Indicating no label
				"xmin": None,
				"ymin": None,
				"xmax": None,
				"ymax": None
			})

	for _, row in labels_df.iterrows():
		all_ids = [i for i in image_folder.glob(f"{row['Image_ID']}*") if i.suffix != ".npy"]
		image_path = image_folder / f"{row['Image_ID']}"
		if all_ids:
			image_path = all_ids[0]

		if image_path.exists():
			img = load_image_(image_path)
			img_width, img_height = img.size

			x_center = row['x_center'] * img_width
			y_center = row['y_center'] * img_height
			width = row['width'] * img_width
			height = row['height'] * img_height

			x_min = x_center - (width / 2)
			y_min = y_center - (height / 2)
			x_max = x_center + (width / 2)
			y_max = y_center + (height / 2)

			converted_bboxes.append({
				"Image_ID": row['Image_ID'],
				"class_id": row['class_id'],
				"xmin": x_min,
				"ymin": y_min,
				"xmax": x_max,
				"ymax": y_max
			})
		else:
			print(f"Image {image_path} not found.")

	return pd.DataFrame(converted_bboxes)

# Example usage
converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax
37,ID_RUW7ek,2,287.000064,57.000112,416.000000,154.000080
74,ID_K6JkTA,1,247.000752,983.998080,1455.000624,2856.998784
90,ID_SCg5VZ,1,200.000256,96.000000,1166.999808,1802.000384
46,ID_TSkjv2,1,-0.000540,2.999835,1041.999660,483.999705
13,ID_vDdYaf,1,17.000189,94.000640,795.000611,1280.000000


In [17]:
converted_labels.describe()

,class_id,xmin,ymin,xmax,ymax
count,95.000000,95.000000,95.000000,95.000000,95.000000
mean,1.126316,358.452658,331.631492,1269.147347,1588.589391
std,0.775175,538.727657,481.446497,950.695543,1184.230438
min,0.000000,-0.002016,-0.001632,58.999824,53.000064
25%,1.000000,1.000392,20.500032,498.499968,704.499664
50%,1.000000,114.999840,114.000000,960.000000,1195.000320
75%,2.000000,477.500328,513.499776,1862.000208,2442.499248
max,2.000000,2514.001344,2346.998784,3897.998496,4128.002064


In [18]:
converted_labels["Image_ID"].nunique()

56

In [19]:
converted_labels['class_id'].value_counts()

class_id
1    37
2    35
0    23
Name: count, dtype: int64

In [20]:
class_map

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [21]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

class
cssvd          37
healthy        35
anthracnose    23
Name: count, dtype: int64

In [22]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
6,ID_vdbf6i,1,41.999520,33.000320,777.999840,1280.000640,cssvd
68,ID_Q9wpbL,2,0.000000,594.000288,2948.998752,3571.999200,healthy
55,ID_xOnGt4,1,15.000000,64.000000,949.999680,1280.000000,cssvd
56,ID_v1PbJC,0,12.000096,-0.001632,1666.999872,2470.999776,anthracnose
32,ID_ET34jY,0,0.000000,722.000000,916.999200,3602.000000,anthracnose


In [23]:
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
6347,ID_dA1GQh.jpg,cssvd,0.000011,0.000000,270.257233,63.317005,658.994263
5393,ID_RUW7ek.jpg,healthy,0.000011,81.325630,109.069336,106.974579,147.528030
6732,ID_SCg5VZ.jpg,anthracnose,0.000005,48.254234,596.971436,221.165894,1128.415527
1751,ID_u0KnR0.jpg,anthracnose,0.000133,1627.880737,1031.949829,2029.493042,1250.120239
6675,ID_SCg5VZ.jpg,cssvd,0.000019,452.797058,214.502563,847.373474,694.955444


In [24]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
7781,ID_OB1kf9,anthracnose,0.000001,296.825500,1.367874,601.409973,110.946617
5343,ID_RUW7ek,healthy,0.000041,0.069998,40.713417,153.333832,325.309631
6911,ID_sFXWTT,healthy,0.005014,202.145584,313.144623,289.054626,401.223907


In [25]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels["Image_ID"].unique()}

In [26]:
converted_labels.isna().sum()

Image_ID    0
class_id    0
xmin        0
ymin        0
xmax        0
ymax        0
class       0
dtype: int64

In [27]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

56

In [28]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [29]:
sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
5036,ID_YQlY8q,cssvd,0.000010,875.265808,540.819702,1334.419678,876.944092
5575,ID_Xaw09O,cssvd,0.000036,1238.498901,175.694885,1279.950684,673.339844
3997,ID_wFa3gD,healthy,0.000004,1344.084595,2448.000000,1770.438721,2448.000000


In [30]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
4010,ID_wFa3gD,healthy,0.000003,0.000000,1172.713623,58.441364,1596.534424,2
5509,ID_yZNP5q,cssvd,0.000004,272.330353,86.611717,389.779205,206.331604,1
974,ID_t1Jamz,anthracnose,0.000003,776.671326,3920.054443,1421.386963,4032.000000,0


In [31]:
predictions = get_preds_data(sub, None)

len(predictions)

56

In [32]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

Evaluation metric at: 0.0  score : {'TP': 93, 'FP': 8307, 'FN': 2, 'Precision': 0.01107142857142857, 'Recall': 0.9789473684210527, 'F1 Score': 0.02189523248969982, 'Accuracy': 0.011068793144489407}
Evaluation metric at: 0.06785714285714285  score : {'TP': 84, 'FP': 58, 'FN': 11, 'Precision': 0.5915492957746479, 'Recall': 0.8842105263157894, 'F1 Score': 0.7088607594936709, 'Accuracy': 0.5490196078431373}
Evaluation metric at: 0.1357142857142857  score : {'TP': 82, 'FP': 43, 'FN': 13, 'Precision': 0.656, 'Recall': 0.8631578947368421, 'F1 Score': 0.7454545454545454, 'Accuracy': 0.5942028985507246}
Evaluation metric at: 0.20357142857142857  score : {'TP': 77, 'FP': 34, 'FN': 18, 'Precision': 0.6936936936936937, 'Recall': 0.8105263157894737, 'F1 Score': 0.7475728155339806, 'Accuracy': 0.5968992248062015}
Evaluation metric at: 0.2714285714285714  score : {'TP': 74, 'FP': 28, 'FN': 21, 'Precision': 0.7254901960784313, 'Recall': 0.7789473684210526, 'F1 Score': 0.751269035532995, 'Accuracy': 0.

Evaluation metric at: 0.40714285714285714  score : {'TP': 65, 'FP': 14, 'FN': 30, 'Precision': 0.8227848101265823, 'Recall': 0.6842105263157895, 'F1 Score': 0.7471264367816093, 'Accuracy': 0.5963302752293578}
Evaluation metric at: 0.475  score : {'TP': 61, 'FP': 12, 'FN': 34, 'Precision': 0.8356164383561644, 'Recall': 0.6421052631578947, 'F1 Score': 0.7261904761904762, 'Accuracy': 0.5700934579439252}
Evaluation metric at: 0.5428571428571428  score : {'TP': 57, 'FP': 8, 'FN': 38, 'Precision': 0.8769230769230769, 'Recall': 0.6, 'F1 Score': 0.7124999999999999, 'Accuracy': 0.5533980582524272}
Evaluation metric at: 0.6107142857142857  score : {'TP': 52, 'FP': 4, 'FN': 43, 'Precision': 0.9285714285714286, 'Recall': 0.5473684210526316, 'F1 Score': 0.6887417218543047, 'Accuracy': 0.5252525252525253}
Evaluation metric at: 0.6785714285714285  score : {'TP': 45, 'FP': 3, 'FN': 50, 'Precision': 0.9375, 'Recall': 0.47368421052631576, 'F1 Score': 0.6293706293706294, 'Accuracy': 0.45918367346938777}


In [33]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
50,ID_Hm1rUc,1,478.000656,382.997664,3016.001520,2441.998944,cssvd
24,ID_XSYdRk,1,0.000000,0.000000,616.000140,1080.000000,cssvd
33,ID_QEGas6,0,174.999840,316.999680,735.999840,1280.000000,anthracnose
83,ID_yZDVIT,2,331.000176,1.999920,416.000208,227.999824,healthy
18,ID_ydI3Pa,0,0.000000,60.999264,829.999296,916.000800,anthracnose


In [34]:
import torch
from torchmetrics.detection import MeanAveragePrecision


def compute_map(preds, targets):
    # Initialize the metric
    metric = MeanAveragePrecision()

    # Update metric with predictions and targets
    metric.update(preds, targets)

    # Compute the results
    result = metric.compute()

    return result


thrs = np.linspace(0.000001, 0.95, 15)
for i in thrs:
    preds = list(get_preds_data(sub, i).values())

    targets = list(ground_truth.values())

    # Compute mAP
    results = compute_map(preds, targets)

    # Print results
    print("mAP Results:", i, " - ", results)

/data/home/eak/learning/nganga_ai/AminiCocoa/Amini-Cocoa-Contamination-Challenge/.venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)


mAP Results: 1e-06  -  {'map': tensor(0.5317), 'map_50': tensor(0.8021), 'map_75': tensor(0.5960), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.5328), 'mar_1': tensor(0.4339), 'mar_10': tensor(0.6212), 'mar_100': tensor(0.6423), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.6423), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.06785807142857142  -  {'map': tensor(0.5238), 'map_50': tensor(0.7853), 'map_75': tensor(0.5920), 'map_small': tensor(-1.), 'map_medium': tensor(-1.), 'map_large': tensor(0.5243), 'mar_1': tensor(0.4339), 'mar_10': tensor(0.6091), 'mar_100': tensor(0.6091), 'mar_small': tensor(-1.), 'mar_medium': tensor(-1.), 'mar_large': tensor(0.6091), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.13571514285714284  -  {'map': tensor(0.5190), 'map_50': tensor(

In [35]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [36]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
1867,ID_VBvSm2,healthy,0.000011,20.728870,532.406921,179.517242,574.678650,2
1823,ID_VBvSm2,cssvd,0.000088,1230.735596,74.794083,1280.000000,228.380966,1
2690,ID_yZDVIT,healthy,0.000004,356.369598,53.851318,401.381805,144.864822,2
7626,ID_xOnGt4,cssvd,0.000003,1114.397705,960.000000,1266.106079,960.000000,1
6946,ID_sFXWTT,healthy,0.000255,215.050537,156.320740,416.000000,411.724487,2
